# 基于多模态多特征的证券市场股票涨跌情况预测任务

在本任务中，我们通过华为开源自研AI框架MindSpore搭建起了基于多模态多特征的深度学习模型，从图像（img）和文本（txt）两种信息载体出发，结合市场主体消息、股票价格呈现、股票技术分析等多种渠道，对证券市场股票（以上证50指数为例）的涨跌情况进行预测性分析。


## 模型简介

在模型设计上，我们主要基于vit-transformer和lstm两种模型构建，对图像和文本数据集进行结构化处理并输入到多模态模型中进行训练。



## 数据处理

开始实验之前，请确保本地已经安装了Python环境并安装了MindSpore框架和MindSpore Vision套件。

### 数据准备


本文研究需要获取上证50指数在每个交易日分别以日和以分钟为频率的历史交易数据，以及上证50指数成分股相关的新闻文本数据，以上数据的时间区间为2012年1月1日至2022年4月25日。

表 5 上证50指数数据集统计
| |训练集|训练集|训练集|测试集|测试集|测试集|
|-|-|-|-|-|-|-|
| |涨|跌|合计|涨|跌|合计|
|交易行情图|1017|987|2004|258|243|501|
|相关新闻天数|958|938|1896|254|240|494|

本文采用的数据集包括两个部分：上证50指数历史交易数据和财经金融新闻文本数据。
#### 图像数据
其中，我们从聚宽平台获取2012年1月4日~2022年4月25日的股票交易数据，共2505个交易日的日频与分钟频的历史数据，其中前2004个交易日的数据所生成的折线图为训练集，后501个交易日的数据所生成的折线图为测试集。

* 股票收盘价折现图即交易行情图的生成。基于股市K线图的观测原理，本文使用收盘价折线图作为股市的图形特征输入模型。文本通过使用聚宽平台开源的Python财经数据接口获取上证50指数的分钟线数据，根据分钟数据生成n天的股票收盘价折线图，并通过将背景色设置为黑色、缩小画布周边无效部分面积等方式，增加图中有效信息面积的占比。

![图像部分信息数据](images/img_example.jpg)

图 6 上证50指数连续14天收盘价折线图（2012-01-04至2012-01-30）

通过特殊数据处理和加工手段，将证券市场动向、技术分析、价格情况等信息要素以时间为排列，构建起图像部分数据。并通过图像形式融合“上证50”的证券价格相关特征，形成模型图片部分输入内容（x）.
以图6为例，折线图以一个交易日的时间（单位：分钟）为x轴，以收盘价价格（单位：元）为y轴，图中14条彩色的折线分别表示连续14天的收盘价变动情况。



- 数据集大小：共2505个交易日的日频与分钟频的历史数据
    - 训练集：前2004个交易日的数据所生成的折线图
    - 测试集：后501个交易日的数据所生成的折线图
- 数据格式：RGB

#### 文本数据

![文本部分信息数据](images/txt_example.png)

通过专业财经类信息数据库（国泰安研究服务中心CSMAR系列数据库），经过相关性筛选和内容甄别，获取到《上海证券报》中每交易日开盘前的关于上证50指数成分股的相关报道，构建起文本部分数据。通过序列形式表达“上证50”证券的市场主体消息特征，形成模型文本部分输入内容（y）.

![文本部分信息数据2](images/txt_example2.png)

数据集路径结构如下。?
 ```bash
└─newdataset/
    ├─img/                
    └─text/
        └─log/  
    
```


### 数据预处理及数据加载

实验选择上证50指数作为股指预测的研究对象，其中基本技术分析数据主要使用的是上证50指数每天的交易行情数据，如当天的成交量、最高价、最低价、昨收价、涨跌额、涨跌幅、成交额、开盘价、收盘价等，本项目选择以当天的收盘价为交易行情数据。

表 1 未经处理的部分行情数据
| 日期 | 开盘价 | 收盘价 | 最高价 | 最低价 | 成交量 | 成交额 |
| --- | --- | --- | --- | --- | --- | --- |
|2012/1/4 | 1628.17 | 1595.12 | 1631.26 | 1594.42 | 1282270500 | 11591786058|
|2012/1/5 | 1591.24 | 1596.59 | 1622.84 | 1590.54 | 1807836500 | 15109105146|
|2012/1/6 | 1595.73 | 1608.36 | 1609.21 | 1588.6 | 1386997300 | 11459377051|
|2012/1/9 | 1608.49 | 1659.05 | 1659.58 | 1598.39 | 2162757900 | 20009493676|
|2012/1/10 | 1658.2 | 1704.74 | 1707.6 | 1655.3 | 2851741700 | 26442594973|
|2012/1/11 | 1703.65 | 1694.26 | 1709.89 | 1686.64 | 1843090600 |18803937162|

这些数据可以通过使用聚宽平台开源的Python财经数据接口来获取。首先利用接口获得上证50指数在时间区间范围内的日频行情数据，生成导出csv文件后，将日期数据与对应的收盘价数据保留，再使公式（16）计算第二天收盘价相比较于第一天的收盘价的涨跌幅度，若大于或者等于0则为涨势，标签记为1；若小于0则为跌势，标签记为0。

其中change为收盘价的涨跌幅度，closei为第一天的收盘价，closei + 1为第二天的收盘价。

表 2 以收盘价为基本技术分析数据的部分行情数据
| 日期 | 收盘价 | 标签 |
| --- | --- | --- |
|2012/1/4 | 1595.12 | - |
|2012/1/5 | 1596.59 | 1 |
|2012/1/6 | 1608.36 | 1 |
|2012/1/9 | 1659.05 | 1 |
|2012/1/10 | 1704.74 | 1 |
|2012/1/11 | 1694.26 | 0 |


#### 图片数据

借助Dataset中map方法，实现对img列的数据预处理，将图像通道由H-W-C转变为C-H-W，并实现裁剪等图像操作。

In [26]:
from PIL import Image
import numpy as np
import os
import mindspore.dataset as ds

def get_all_files(file_path,image_size):
    """
    获取图片路径及其标签
    :param file_path: a sting, 图片所在目录
    :param is_random: True or False, 是否乱序
    :return:
    """
    image_list = []
    label_list = []
    print("loading data start....")
    for item in os.listdir(file_path):
        item_path = file_path + '/' + item
        item_label = item.split('：')[1]
        item_label=int(item_label.split('.')[0])
        
        img = Image.open(item_path)
        arr = np.asarray(img, dtype="float32")
        arr.resize((image_size, image_size, 3),refcheck=False)
        arr=arr/255
        image_list.append(arr)
        if(item_label==-1):
            label_list.append(0)
        else:
            label_list.append(1)
        # label_list.append(item_label)

    label_list=np.array(label_list)
    image_list=np.array(image_list)
    print("loading data ok....")
    print("lable_list:"+str(label_list[0:5]))
    print("images_list:"+str(image_list[0]))
    return image_list, label_list

In [1]:
def infer_transform(dataset, resize):

    mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    std = [0.229 * 255, 0.224 * 255, 0.225 * 255]

#     trans = [
#              c_transforms.Resize([resize, resize]),
#              c_transforms.Normalize(mean=mean, std=std),
#              c_transforms.HWC2CHW()]
    trans = [c_transforms.HWC2CHW()]

    dataset = dataset.map(operations=trans,
                          input_columns="img",
                          num_parallel_workers=1)

    return dataset


#### 文本数据

通过词典构建、词向量表示、短文本填充和长文本裁剪等文本处理手段对不定长文本进行数据预处理，并构建起文本的预训练词向量表示。

In [2]:
#text/load_data
import numpy as np
import pandas as pd
import pickle
import numpy as np
from collections import Counter
from itertools import accumulate
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.sans-serif']=['SimHei']

BASE_DIR = "newdataset/text/log"

KERAS_MODEL_SAVE_PATH = '%s/Bi-LSTM-4-NER.h5' % BASE_DIR
WORD_DICTIONARY_PATH = '%s/word_dictionary.pk' % BASE_DIR
InVERSE_WORD_DICTIONARY_PATH = '%s/inverse_word_dictionary.pk' % BASE_DIR
LABEL_DICTIONARY_PATH = '%s/label_dictionary.pk' % BASE_DIR
OUTPUT_DICTIONARY_PATH = '%s/output_dictionary.pk' % BASE_DIR

CONSTANTS = [
             KERAS_MODEL_SAVE_PATH,
             InVERSE_WORD_DICTIONARY_PATH,
             WORD_DICTIONARY_PATH,
             LABEL_DICTIONARY_PATH,
             OUTPUT_DICTIONARY_PATH
             ]


def load_data():
    data = pd.read_csv("newdataset/text/total.csv")

    text=data['Title']
    input_data = list()

    number=0
    for i in range(0,len(text)-1):
        for j in range(1, len(text[i]) - 1):
            text_data = list()
            text_data.append(number)
            text_data.append(text[i][j])
            input_data.append(text_data)
        number+=1
    input_data = pd.DataFrame(input_data,columns=['number', 'newColumn'])
    input_data=input_data[['number', 'newColumn']]
    return input_data



# 数据查看
def data_review():

    # 数据导入
    input_data = load_data()

    # 基本的数据review
    sent_num = input_data['number'].astype(np.int).max()
    print("一共有%s个句子。\n"%sent_num)

    vocabulary = input_data['newColumn'].unique()
    print("一共有%d个单词。"%len(vocabulary))
    print("前10个单词为：%s.\n"%vocabulary[:11])

    # pos_arr = input_data['pos'].unique()
    # print("单词的词性列表：%s.\n"%pos_arr)
    #
    # ner_tag_arr = input_data['tag'].unique()
    # print("NER的标注列表：%s.\n" % ner_tag_arr)

    df = input_data[['number', 'newColumn']].groupby('number').count()
    sent_len_list = df['newColumn'].tolist()
    print("句子长度及出现频数字典：\n%s." % dict(Counter(sent_len_list)))

    # 绘制句子长度及出现频数统计图
    sort_sent_len_dist = sorted(dict(Counter(sent_len_list)).items(), key=itemgetter(0))
    sent_no_data = [item[0] for item in sort_sent_len_dist]
    sent_count_data = [item[1] for item in sort_sent_len_dist]
    plt.bar(sent_no_data, sent_count_data)
    plt.title("句子长度及出现频数统计图")
    plt.xlabel("句子长度")
    plt.ylabel("句子长度出现的频数")
    plt.savefig("%s/句子长度及出现频数统计图.png" % BASE_DIR)
    plt.show()

    # 绘制句子长度累积分布函数(CDF)
    sent_pentage_list = [(count/sent_num) for count in accumulate(sent_count_data)]

    # 寻找分位点为quantile的句子长度
    quantile = 0.95
    print(list(sent_pentage_list))
    index=0
    for length, per in zip(sent_no_data, sent_pentage_list):
        if per>=quantile:
            index = length
            break
    print("\n分位点为%s的句子长度:%d." % (quantile, index))

    # 绘制CDF
    plt.plot(sent_no_data, sent_pentage_list)
    plt.hlines(quantile, 0, index, colors="c", linestyles="dashed")
    plt.vlines(index, 0, quantile, colors="c", linestyles="dashed")
    plt.text(0, quantile, str(quantile))
    plt.text(index, 0, str(index))
    plt.title("句子长度累积分布函数图")
    plt.xlabel("句子长度")
    plt.ylabel("句子长度累积频率")
    plt.savefig("%s/句子长度累积分布函数图.png" % BASE_DIR)
    plt.show()

# 数据处理
def data_processing():
    # 数据导入
    input_data = load_data()
    print("load data over.....")

    # 标签及词汇表
    labels, vocabulary = list(input_data['number'].unique()), list(input_data['newColumn'].unique())

    # 字典列表
    word_dictionary = {word: i+1 for i, word in enumerate(vocabulary)}
    inverse_word_dictionary = {i+1: word for i, word in enumerate(vocabulary)}
    # label_dictionary = {label: i+1 for i, label in enumerate(labels)}
    # output_dictionary = {i+1: labels for i, labels in enumerate(labels)}

    dict_list = [word_dictionary, inverse_word_dictionary]

    # 保存为pickle形式
    for dict_item, path in zip(dict_list, CONSTANTS[1:]):
        with open(path, 'wb') as f:
            pickle.dump(dict_item, f)

# if __name__ == '__main__':
#     load_data()
#     data_review()
   

#### 数据集结构化生成

通过mindspore框架中GeneratorDataset（）方法，构建起[“img”,“txt”,“lable”]的三元数据集Dataset。

其中，通过自定义IMDBData类，生成关于{img,txt,lable}的可迭代对象，用于GeneratorDataset（）方法中source传参。

In [6]:
#IMDBData
import numpy as np


class IMDBData():
    def __init__(self,img,text,lable):
        self.text=[]
        self.lable=[]
        self.img=[]
        for i in range(0,len(lable)):
            self.text.append(text[i])
            self.lable.append(lable[i])
            self.img.append(img[i])
    def __getitem__(self, idx):
        return  self.img[idx],self.text[idx],self.lable[idx]

    def __len__(self):
        return len(self.lable)

### 数据加载

通过数据集加载接口加载数据集。

In [27]:
    input_shape= 1625
    resize=224
    batch_size=15
    test_dir="newdataset/img"
    #数据读入
    txt, vocab_size, inverse_word_dictionary = input_data_for(input_shape)
    image_train_batch, label_train_batch= get_all_files(test_dir, resize)
    label_train_batch= np.eye(2, dtype=np.uint8)[label_train_batch]

    if(len(image_train_batch)>len(txt)):
        size=len(txt)
    else:
        size=len(image_train_batch)
    end=int(size*0.8)

    train_x_txt=txt[0:end]
    test_x_txt=txt[end:size-1]
    print("train_x_txt:"+str(train_x_txt[0:5]))

    train_x_img,train_y=image_train_batch[0:end],label_train_batch[0:end]
    test_x_img,test_y=image_train_batch[end:size-1],label_train_batch[end:size-1]
    print("train_lable_one_hot:"+str(train_y[0:5]))
    datesets=[]
    for i in range(0,end,batch_size):
        datesets.append([train_x_img[i:i+batch_size],train_x_txt[i:i+batch_size],train_y[i:i+batch_size]])
    # # column_names = ["img","txt","lable"]
    dataset = ds.GeneratorDataset(source=IMDBData(train_x_img,train_x_txt,train_y),column_names=["img","txt","lable"])
    dataset = infer_transform(dataset, resize)
    dataset = dataset.batch(batch_size)

load data over.....
len(x)3126
len(x[0])1625
vocab_size3506
loading data start....
loading data ok....
lable_list:[0 0 1 0 1]
images_list:[[[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 ...

 [[0.02745098 0.         0.        ]
  [0.00784314 0.01176471 0.        ]
  [0.         0.01568628 0.        ]
  ...
  [0.         0.         0.        ]
  [0.       

## 构建网络



### vit-transformer 模型

借助于mindspore框架，依次完成PatchEmbedding、Attention、TransformerEncoder等Vision Transformer（ViT）模型的基础模块类


#### Attention模块

核心内容是为输入向量的每个单词学习一个权重。通过给定一个任务相关的查询向量Query向量，计算Query和各个Key的相似性或者相关性得到注意力分布，即得到每个Key对应Value的权重系数，然后对Value进行加权求和得到最终的Attention数值。

In [10]:
from mindspore import nn, ops
import mindspore as ms
class Attention(nn.Cell):
    def __init__(self,
                 dim: int,
                 num_heads: int = 8,
                 keep_prob: float = 1.0,
                 attention_keep_prob: float = 1.0):
        super(Attention, self).__init__()

        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = ms.Tensor(head_dim ** -0.5)

        self.qkv = nn.Dense(dim, dim * 3)
        self.attn_drop = nn.Dropout(attention_keep_prob)
        self.out = nn.Dense(dim, dim)
        self.out_drop = nn.Dropout(keep_prob)

        self.mul = ops.Mul()
        self.reshape = ops.Reshape()
        self.transpose = ops.Transpose()
        self.unstack = ops.Unstack(axis=0)
        self.attn_matmul_v = ops.BatchMatMul()
        self.q_matmul_k = ops.BatchMatMul(transpose_b=True)
        self.softmax = nn.Softmax(axis=-1)

    def construct(self, x):
        """Attention construct."""
        b, n, c = x.shape

        # 最初的输入向量首先会经过Embedding层映射成Q(Query)，K(Key)，V(Value)三个向量
        # 由于是并行操作，所以代码中是映射成为dim*3的向量然后进行分割
        qkv = self.qkv(x)

        #多头注意力机制就是将原本self-Attention处理的向量分割为多个Head进行处理
        qkv = self.reshape(qkv, (b, n, 3, self.num_heads, c // self.num_heads))
        qkv = self.transpose(qkv, (2, 0, 3, 1, 4))
        q, k, v = self.unstack(qkv)

        # 自注意力机制的自注意主要体现在它的Q，K，V都来源于其自身
        # 也就是该过程是在提取输入的不同顺序的向量的联系与特征
        # 最终通过不同顺序向量之间的联系紧密性（Q与K乘积经过Softmax的结果）来表现出来
        attn = self.q_matmul_k(q, k)
        attn = self.mul(attn, self.scale)
        attn = self.softmax(attn)
        attn = self.attn_drop(attn)

        # 其最终输出则是通过V这个映射后的向量与QK经过Softmax结果进行weight sum获得
        # 这个过程可以理解为在全局上进行自注意表示
        out = self.attn_matmul_v(attn, v)
        out = self.transpose(out, (0, 2, 1, 3))
        out = self.reshape(out, (b, n, c))
        out = self.out(out)
        out = self.out_drop(out)

        return out

   #### PatchEmbedding模块

在ViT模型中：

通过将输入图像在每个channel上划分为16*16个patch，这一步是通过卷积操作来完成的，当然也可以人工进行划分，但卷积操作也可以达到目的同时还可以进行一次而外的数据处理；例如一幅输入224 x 224的图像，首先经过卷积处理得到16 x 16个patch，那么每一个patch的大小就是14 x 14。

再将每一个patch的矩阵拉伸成为一个1维向量，从而获得了近似词向量堆叠的效果。上一步得到的14 x 14的patch就转换为长度为196的向量。

In [11]:
class PatchEmbedding(nn.Cell):
    MIN_NUM_PATCHES = 4
    def __init__(self,
                 image_size: int = 224,
                 patch_size: int = 16,
                 embed_dim: int = 768,
                 input_channels: int = 3):
        super(PatchEmbedding, self).__init__()

        self.image_size = image_size
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2

        # 通过将输入图像在每个channel上划分为16*16个patch
        self.conv = nn.Conv2d(input_channels, embed_dim, kernel_size=patch_size, stride=patch_size, has_bias=True)
        self.reshape = P.Reshape()
        self.transpose = P.Transpose()

    def construct(self, x):
        """Path Embedding construct."""
        x = self.conv(x)
        b, c, h, w = x.shape

        # 再将每一个patch的矩阵拉伸成为一个1维向量，从而获得了近似词向量堆叠的效果；
        x = self.reshape(x, (b, c, h * w))
        x = self.transpose(x, (0, 2, 1))

        return x

#### Transformer Encoder

在了解了Self-Attention结构之后，通过与Feed Forward，Residual Connection等结构的拼接就可以形成Transformer的基础结构，接下来就利用Self-Attention来构建ViT模型中的TransformerEncoder部分，类似于构建了一个Transformer的编码器部分，将TransformerEncoder结构和一个多层感知器（MLP）结合，就构成了ViT模型的backbone部分。


In [12]:
import  mindspore as ms
import mindspore.nn as nn
import mindspore.train.callback as callback
from mindvision.classification.models import FeedForward, ResidualCell
class TransformerEncoder(nn.Cell):
    def __init__(self,
                 dim: int,
                 num_layers: int,
                 num_heads: int,
                 mlp_dim: int,
                 keep_prob: float = 1.,
                 attention_keep_prob: float = 1.0,
                 drop_path_keep_prob: float = 1.0,
                 activation: nn.Cell = nn.GELU,
                 norm: nn.Cell = nn.LayerNorm):
        super(TransformerEncoder, self).__init__()
        layers = []

        # 从vit_architecture图可以发现，多个子encoder的堆叠就完成了模型编码器的构建
        # 在ViT模型中，依然沿用这个思路，通过配置超参数num_layers，就可以确定堆叠层数
        for _ in range(num_layers):
            normalization1 = norm((dim,))
            normalization2 = norm((dim,))
            attention = Attention(dim=dim,
                                  num_heads=num_heads,
                                  keep_prob=keep_prob,
                                  attention_keep_prob=attention_keep_prob)

            feedforward = FeedForward(in_features=dim,
                                      hidden_features=mlp_dim,
                                      activation=activation,
                                      keep_prob=keep_prob)

            # ViT模型中的基础结构与标准Transformer有所不同
            # 主要在于Normalization的位置是放在Self-Attention和Feed Forward之前
            # 其他结构如Residual Connection，Feed Forward，Normalization都如Transformer中所设计
            layers.append(
                nn.SequentialCell([
                    # Residual Connection，Normalization的结构可以保证模型有很强的扩展性
                    # 保证信息经过深层处理不会出现退化的现象，这是Residual Connection的作用
                    # Normalization和dropout的应用可以增强模型泛化能力
                    ResidualCell(nn.SequentialCell([normalization1,
                                                    attention])),

                    ResidualCell(nn.SequentialCell([normalization2,
                                                    feedforward]))
                ])
            )
        self.layers = nn.SequentialCell(layers)

    def construct(self, x):
        """Transformer construct."""
        return self.layers(x)

#### 整体构建ViT

以下代码构建了一个完整的ViT模型。

In [13]:
from typing import Optional

class ViT(nn.Cell):
    def __init__(self,
                 image_size: int = 224,
                 input_channels: int = 3,
                 patch_size: int = 16,
                 embed_dim: int = 768,
                 num_layers: int = 12,
                 num_heads: int = 12,
                 mlp_dim: int = 3072,
                 keep_prob: float = 1.0,
                 attention_keep_prob: float = 1.0,
                 drop_path_keep_prob: float = 1.0,
                 activation: nn.Cell = nn.GELU,
                 norm: Optional[nn.Cell] = nn.LayerNorm,
                 pool: str = 'cls') -> None:
        super(ViT, self).__init__()

        self.patch_embedding = PatchEmbedding(image_size=image_size,
                                              patch_size=patch_size,
                                              embed_dim=embed_dim,
                                              input_channels=input_channels)
        num_patches = self.patch_embedding.num_patches

        # 此处增加class_embedding和pos_embedding，如果不是进行分类任务
        # 可以只增加pos_embedding，通过pool参数进行控制
        self.cls_token = init(init_type=Normal(sigma=1.0),
                              shape=(1, 1, embed_dim),
                              dtype=ms.float32,
                              name='cls',
                              requires_grad=True)

        # pos_embedding也是一组可以学习的参数，会被加入到经过处理的patch矩阵中
        self.pos_embedding = init(init_type=Normal(sigma=1.0),
                                  shape=(1, num_patches + 1, embed_dim),
                                  dtype=ms.float32,
                                  name='pos_embedding',
                                  requires_grad=True)

        # axis=1定义了会在向量的开头加入class_embedding
        self.concat = P.Concat(axis=1)

        self.pool = pool
        self.pos_dropout = nn.Dropout(keep_prob)
        self.norm = norm((embed_dim,))
        self.tile = P.Tile()
        self.transformer = TransformerEncoder(dim=embed_dim,
                                              num_layers=num_layers,
                                              num_heads=num_heads,
                                              mlp_dim=mlp_dim,
                                              keep_prob=keep_prob,
                                              attention_keep_prob=attention_keep_prob,
                                              drop_path_keep_prob=drop_path_keep_prob,
                                              activation=activation,
                                              norm=norm)

    def construct(self, x):
        """ViT construct."""
        x = self.patch_embedding(x)

        # class_embedding主要借鉴了BERT模型的用于文本分类时的思想
        # 在每一个word vector之前增加一个类别值，通常是加在向量的第一位
        cls_tokens = self.tile(self.cls_token, (x.shape[0], 1, 1))
        x = self.concat((cls_tokens, x))
        x += self.pos_embedding

        x = self.pos_dropout(x)
        x = self.transformer(x)
        x = self.norm(x)

        # 增加的class_embedding是一个可以学习的参数，经过网络的不断训练
        # 最终以输出向量的第一个维度的输出来决定最后的输出类别；
        x = x[:, 0]

        return x

### Bi- LSTM模型

使用mindspore框架中LSTM实现双向LSTM

In [14]:
#Bi_LSTM_model
import pickle

import keras
import numpy as np
import pandas as pd
import keras.backend as K
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Dropout, Masking
from keras.utils import np_utils

from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, f1_score, confusion_matrix

#from text.load_data import CONSTANTS, load_data, data_processing, BASE_DIR




def input_data_for_model(input_shape):

    # 数据导入
    input_data = load_data()
    # 数据处理
    data_processing()
    # 导入字典
    with open(CONSTANTS[1], 'rb') as f:
        word_dictionary = pickle.load(f)
    with open(CONSTANTS[2], 'rb') as f:
        inverse_word_dictionary = pickle.load(f)
    with open(CONSTANTS[3], 'rb') as f:
        label_dictionary = pickle.load(f)
    with open(CONSTANTS[4], 'rb') as f:
        output_dictionary = pickle.load(f)
    vocab_size = len(word_dictionary.keys())
    label_size = len(label_dictionary.keys())

    # 处理输入数据
    aggregate_function = lambda input: [(word, label) for word,label in
                                            zip(input['word'].values.tolist(),
                                                input['tag'].values.tolist())]

    grouped_input_data = input_data.groupby('sent_no').apply(aggregate_function)
    sentences = [sentence for sentence in grouped_input_data]

    x = [[word_dictionary[word[0]] for word in sent] for sent in sentences]
    x = pad_sequences(maxlen=input_shape, sequences=x, padding='post', value=0)
    y = [[label_dictionary[word[1]] for word in sent] for sent in sentences]
    y = pad_sequences(maxlen=input_shape, sequences=y, padding='post', value=0)
    y = [np_utils.to_categorical(label, num_classes=label_size+1) for label in y]
    return x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary

def draw_train(history):
        
        plt.plot(history.history['accuracy'],'r-')
        plt.plot(history.history['val_accuracy'],'b:')
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train','Test'], loc='upper left')
        plt.show()
       
        plt.plot(history.history['loss'],'r-')
        plt.plot(history.history['val_loss'],'b:')
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train','Test'], loc='upper left')
        plt.show()

# # 定义深度学习模型：Bi-LSTM
def create_Bi_LSTM(vocab_size, label_size, input_shape, output_dim, n_units, out_act, activation):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size + 1,
                        output_dim=output_dim,
                        input_length=input_shape,
                        trainable=True,
                        mask_zero=True))
    model.add(LSTM(units=100,input_shape=(1500,128)))
   
    keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
   
    keras.utils.plot_model(model, 'LSTM.png', show_shapes=True)
    model.summary()
    return model


# 模型训练
def model_train():
    # 将数据集分为训练集和测试集，占比为9:1
    input_shape = 1500
    x, vocab_size, inverse_word_dictionary = input_data_for(input_shape)
  
    activation = 'selu'
    out_act = 'softmax'
    n_units = 128
    batch_size = 186
    epochs = 10
    output_dim = 128
    # 模型训练
    lstm_model = create_Bi_LSTM(vocab_size, 19, input_shape, output_dim, n_units, out_act, activation)
    lstm_model.summary()
  
  




def model_test():
    # 导入字典
    with open(CONSTANTS[1], 'rb') as f:
        word_dictionary = pickle.load(f)
    with open(CONSTANTS[4], 'rb') as f:
        output_dictionary = pickle.load(f)

    x, y, output_dictionary, vocab_size, label_size, inverse_word_dictionary=input_data_for_model(40,"D:\\conll\\data\\CoNLL-2003\\eng.testa")
    model_save_path = CONSTANTS[0]
    lstm_model = load_model(model_save_path)
    y_predict=lstm_model.predict(x)



# if __name__ == '__main__':
#     model_train()
#     # input_data_for(input_shape=1500)





### 整体模型搭建

通过toger函数完成整体模型类的搭建工作，具体在construct中体现模型的多输入特性

In [15]:
#together
from mindspore.common.initializer import Normal
from mindvision.classification.utils import init
#from PatchEmbedding import PatchEmbedding
#from TransformerEncoder import TransformerEncoder
import mindspore as ms
import mindspore.ops as P
from mindspore import nn, ops
from typing import Optional

class toger(nn.Cell):
    def __init__(self,
                 vocab_size,
                 num_tags,
                 image_size,
                 seq_length,
                 embedding_dim: int = 32,
                 hidden_dim: int = 64,
                 padding_idx=0,
                 patch_size : int=16,
                 input_channels: int = 3,
                 embed_dim: int = 768,
                 num_layers: int = 12,
                 num_heads: int = 12,
                 mlp_dim: int = 3072,
                 keep_prob: float = 1.0,
                 attention_keep_prob: float = 1.0,
                 drop_path_keep_prob: float = 1.0,
                 activation: nn.Cell = nn.GELU,
                 norm: Optional[nn.Cell] = nn.LayerNorm,
                 pool: str = 'cls'):
        super(toger,self).__init__()
        self.patch_embedding = PatchEmbedding(image_size=image_size,
                                              patch_size=patch_size,
                                              embed_dim=embed_dim,
                                              input_channels=input_channels)
        num_patches = self.patch_embedding.num_patches

        # 此处增加class_embedding和pos_embedding，如果不是进行分类任务
        # 可以只增加pos_embedding，通过pool参数进行控制
        self.cls_token = init(init_type=Normal(sigma=1.0),
                              shape=(1, 1, embed_dim),
                              dtype=ms.float32,
                              name='cls',
                              requires_grad=True)

        # pos_embedding也是一组可以学习的参数，会被加入到经过处理的patch矩阵中
        self.pos_embedding = init(init_type=Normal(sigma=1.0),
                                  shape=(1, num_patches + 1, embed_dim),
                                  dtype=ms.float32,
                                  name='pos_embedding',
                                  requires_grad=True)

        # axis=1定义了会在向量的开头加入class_embedding
        self.concat = P.Concat(axis=1)

        self.pool = pool
        self.pos_dropout = nn.Dropout(keep_prob)
        self.norm = norm((embed_dim,))
        self.tile = P.Tile()
        self.transformer = TransformerEncoder(dim=embed_dim,
                                              num_layers=num_layers,
                                              num_heads=num_heads,
                                              mlp_dim=mlp_dim,
                                              keep_prob=keep_prob,
                                              attention_keep_prob=attention_keep_prob,
                                              drop_path_keep_prob=drop_path_keep_prob,
                                              activation=activation,
                                              norm=norm)

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.seq_length=seq_length
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Dense(hidden_dim, hidden_dim * 2 , 'he_uniform')
        self.Dense = nn.Dense(hidden_dim*6,hidden_dim * 2,'he_uniform')
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = ops.Sigmoid()

    def construct(self, x,y):

        """ViT construct."""
        x = self.patch_embedding(x)

        # class_embedding主要借鉴了BERT模型的用于文本分类时的思想
        # 在每一个word vector之前增加一个类别值，通常是加在向量的第一位
        cls_tokens = self.tile(self.cls_token, (x.shape[0], 1, 1))
        x = self.concat((cls_tokens, x))
        x += self.pos_embedding

        x = self.pos_dropout(x)
        x = self.transformer(x)
        x = self.norm(x)

        # 增加的class_embedding是一个可以学习的参数，经过网络的不断训练
        # 最终以输出向量的第一个维度的输出来决定最后的输出类别；
        img = x[:, 0]

        y = self.embedding(y)
        y, _ = self.lstm(y, seq_length=self.seq_length)
        text = self.hidden2tag(y)

        toge= self.concat((img, text))

        toge=self.dropout(toge)
        toge=self.Dense(toge)

        return self.sigmoid(toge)


### MyWithLossCell

MyWithLossCell类用于模型与损失的链接（net_with_loss），实现对三元数据集损失计算的适配。

In [16]:
#MyWithLossCell
from mindspore.nn import Cell

class MyWithLossCell(Cell):
   def __init__(self, backbone, loss_fn):
       super(MyWithLossCell, self).__init__(auto_prefix=False)
       self._backbone = backbone
       self._loss_fn = loss_fn

   def construct(self, x, y, label):
       out = self._backbone(x, y)
       return self._loss_fn(out, label)

   @property
   def backbone_network(self):
       return self._backbone

### train_one_epoch

自定义train_one_epoch函数用于单步训练的梯度下降和权重更新

In [17]:
#train
import os
from pathlib import Path
from typing import Tuple

import mindspore
from mindspore.dataset import vision
from mindvision.classification import ImageNet
from mindvision.classification.dataset import Mnist
from tqdm import tqdm

#from MyWithLossCell import MyWithLossCell
#from img.load_data import get_all_files
#from text.Bi_LSTM_Model import input_data_for
#from toge import toger
import  mindspore as ms
import mindspore.nn as nn
from mindvision.engine.callback import LossMonitor
from mindvision.engine.loss import CrossEntropySmooth
#from IMDBData import IMDBData


import mindspore.dataset as ds
import numpy as np
from mindspore.dataset.vision import c_transforms



def train_one_epoch(model, dateset,bitch_size,epoch=0):
    model.set_train()
    total = bitch_size
    loss_total = 0
    step_total = 0
    m=dateset.create_tuple_iterator()
    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        for i in dateset.create_tuple_iterator():
            print(i)
            loss = model(*i)
            loss_total += loss.asnumpy()
            step_total += 1
            t.set_postfix(loss=loss_total/step_total)
            t.update(1)

### evaluate函数

自定义evaluate函数用于训练情况评估，并在训练过程中更新一个“最佳损失情况”变量（best_valid_loss）用于保留最佳模型的模型参数（save_checkpoint）。

In [18]:
def evaluate(model, test_dataset, criterion,bitch_size, epoch=0):
    total = bitch_size
    epoch_loss = 0
    epoch_acc = 0
    step_total = 0
    model.set_train(False)

    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        for i in test_dataset:
            predictions = model(i[0],i[1])
            loss = criterion(predictions, i[2])
            epoch_loss += loss.asnumpy()

            acc = binary_accuracy(predictions.asnumpy(), i[1].asnumpy())
            epoch_acc += acc

            step_total += 1
            t.set_postfix(loss=epoch_loss/step_total, acc=epoch_acc/step_total)
            t.update(1)

    return epoch_loss / total

### 损失函数

选取二分类任务中常用的二分类交叉熵作为损失函数（nn.BCELoss）


 ![二分类交叉熵](images/math.png)
 
 

In [23]:
# 定义损失函数
loss = nn.BCELoss(reduction='mean')


## 模型实现

采用TrainOneStepCell，用于实现单步训练，
采用Adam优化器，
采用mindspore框架的cosine_decay_lr实现动态的学习率调整，提高训练效率和成果。

In [24]:
def binary_accuracy(preds, y):
    """
    计算每个batch的准确率
    """

    # 对预测值进行四舍五入
    rounded_preds = np.around(preds)
    correct = (rounded_preds == y).astype(np.float32)
    acc = correct.sum() / len(correct)
    return acc



def train():
    # 定义超参数
    epoch= 30
    # momentum = 0.9
    
    hidden_dim=128
    num_classes = 2
    

    cache_dir = Path.home() / '.mslog'



    network = toger(vocab_size=1500,
                         num_tags=num_classes,
                         image_size=resize,
                         seq_length=input_shape,
                         hidden_dim=hidden_dim)
    net_with_loss = MyWithLossCell(network, loss)

    lr = nn.cosine_decay_lr(min_lr=float(0),
                            max_lr=0.003,
                            total_step=epoch * batch_size,
                            step_per_epoch=batch_size,
                            decay_epoch=90)

    

    optimizer = nn.Adam(network.trainable_params(), learning_rate=lr)

    train_one_step = nn.TrainOneStepCell(net_with_loss, optimizer)

    best_valid_loss = float('inf')
    ckpt_file_name = os.path.join(cache_dir, 'sentiment-analysis.ckpt')
    # ckpt_callback = mindspore.ModelCheckpoint(prefix='toger', directory='./ViT', config=ckpt_file_name)
    
    # 初始化模型
    model = ms.Model(network, loss_fn=loss, optimizer=optimizer, metrics={"acc"})
    # # 训练
    # train_one_step.train(epoch_size,
    #             dataset,
    #             callbacks= LossMonitor(lr))

    

  

def generator_multi_column():
    for i in range(5):
         return np.array([i]), np.array([[i, i + 1], [i + 2, i + 3]])

def train_data(num):
    x = np.array(np.random.rand(num,1))
    y = x*2 + 1
    return np.concatenate((x,y),axis=1)

    


## 模型训练

## 训练

In [ ]:
from tqdm import tqdm
total = batch_size
print("start train...")
epoch= 30

for epoch in range(epoch):
    with tqdm(total=total) as t:
        t.set_description('Epoch %i' % epoch)
        model.train(epoch_size,
            dataset_train,
            callbacks=[ckpt_callback, LossMonitor(lr)],
            dataset_sink_mode=False)
       

start train...


Epoch 0:   0%|                                                                                  | 0/16 [00:00<?, ?it/s]

Epoch:[  0/ 10], step:[    1/  157], loss:[3.904/3.904], time:123548.419 ms, lr:0.00300
Epoch:[  0/ 10], step:[    2/  157], loss:[7.421/5.663], time:117884.310 ms, lr:0.00300
Epoch:[  0/ 10], step:[    3/  157], loss:[6.310/5.878], time:109111.214 ms, lr:0.00300
Epoch:[  0/ 10], step:[    4/  157], loss:[6.431/6.017], time:121945.406 ms, lr:0.00300
Epoch:[  0/ 10], step:[    5/  157], loss:[2.788/5.371], time:105975.384 ms, lr:0.00300
Epoch:[  0/ 10], step:[    6/  157], loss:[1.926/4.797], time:111043.187 ms, lr:0.00300
Epoch:[  0/ 10], step:[    7/  157], loss:[2.207/4.427], time:111263.181 ms, lr:0.00300
Epoch:[  0/ 10], step:[    8/  157], loss:[2.533/4.190], time:105905.382 ms, lr:0.00300
Epoch:[  0/ 10], step:[    9/  157], loss:[1.874/3.933], time:100148.228 ms, lr:0.00300
Epoch:[  0/ 10], step:[   10/  157], loss:[1.663/3.706], time:92576.630 ms, lr:0.00300
Epoch:[  0/ 10], step:[   11/  157], loss:[1.574/3.512], time:101641.179 ms, lr:0.00300
Epoch:[  0/ 10], step:[   12/  15

## 其他说明

   #### 本次任务由于时间以及电脑设备的局限性，模型未能完全训练完成，但在预期设计以及相关理论研究中，本次任务提出的基于图片和文本两种信息载体建立的多模态多特征的深度学习模型，使用数据集结合市场主体消息、股票价格呈现、股票技术分析等多种渠道，对证券市场股票（以上证50指数为例）的涨跌情况进行预测性分析，可以取得很不错的效果，相较于以往的单模型单类型数据本次任务的设计对于股票预测及量化交易领域具有一定研究及实践意义。
   
   本次任务由中南财经政法大学信管19级涂宇飞，祁青峰，杨理（排名不分先后）三人合作研究完成，其他更多说明详见README.md